#1.data

In [ ]:
import os
os.chdir('/content/drive/MyDrive/mini')

In [ ]:
!pip install subword-nmt

In [ ]:
import pandas as pd
# import torch
# import torchvision
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torch.nn.functional as F
import os
import random
#from stream import BIN_Data_Encoder

In [ ]:
import copy
from time import time
import os
import random

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, \
    precision_score, recall_score, auc
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import collections
import math
import copy
from subword_nmt.apply_bpe import BPE
import codecs
from tqdm.notebook import tqdm

torch.manual_seed(2)
np.random.seed(3)

from model import BIN_Interaction_Flat1, BIN_Interaction_Flat2, BIN_Interaction_Flat3
from stream import BIN_Data_Encoder
from data import Cus_DataLoader

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
valid=pd.read_csv('val.csv')

In [ ]:
newnew=pd.read_csv('result.csv')

In [ ]:
newnew

,index,Label,SMILES,Target Sequence
0,4229228,1,CC(C)C[C@H](NC(=O)[C@H](C)NC(=O)[C@H](CC(O)=O)...,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...
1,1800097,1,C\C(CC[C@H]1[C@](C)(O)C[C@H](O)[C@@H]2[C@]1(C)...,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...
2,533876,0,Cc1ccc(S[C@@H]2O[C@H](CO)[C@H](O)[C@@H]([C@H]2...,MGKKQNKKKVEEVLEEEEEEYVVEKVLDRRVVKGKVEYLLKWKGFS...
3,3797721,1,CN[C@@H](C)C(=O)N[C@H]1CN(CC[C@H]2CC[C@H](N2C1...,MTLHSQSTTSPLFPQISSSWVHSPSEAGLPLGTVTQLGSYQISQET...
4,7849640,0,COC(=O)c1c(O)cc(O)c(Cl)c1CCc1nccn1Cc1cc(Cl)cs1,MADNFSLHDALSGSGNPNPQGWPGAWGNQPAGAGGYPGASYPGAYP...
...,...,...,...,...
1999995,6317901,1,Clc1ccc(CNCCCCCc2cnc[nH]2)cc1,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...
1999996,7454462,0,S=C(NCCCCCc1cnc[nH]1)NC1CCCCC1,MDLSMKKFAVRRFFSVYLRRKSRSKSSSLSRLEEEGVVKEIDISHH...
1999997,6745165,1,COc1cc(ccc1NC(=O)[C@@H]1N[C@@H](CC(C)(C)C)[C@@...,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...
1999998,3741498,1,COc1nc2ccccc2nc1C(=O)Nc1cc(CN2CCCC2)c(O)c(c1)N...,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...


In [ ]:
newnew.index.values

array([      0,       1,       2, ..., 1999997, 1999998, 1999999])

In [ ]:
test

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,DrugBank ID,Gene,Label,SMILES,Target Sequence
0,0,11705,11723,DB00321,P20309,1.0,CN(C)CCC=C1C2=CC=CC=C2CCC2=CC=CC=C12,MTLHNNSTTSPLFPNISSSWIHSPSDAGLPPGTVTHFGSYNVSRAA...
1,1,14450,14472,DB01084,Q8WUI4,0.0,CCOCCN1C(=NC2=CC=CC=C12)N1CCCN(C)CC1,MDLRVGQRPPVEPPPEPTLLALQRPQRLHHHLFLAGLQQQRSVEPM...
2,2,14382,14404,DB00679,P10632,1.0,CSC1=CC2=C(SC3=CC=CC=C3N2CCC2CCCCN2C)C=C1,MEPFVVLVLCLSFMLLFSLWRQSCRRRKLPPGPTPLPIIGNMLQID...
3,3,1388,1391,DB01337,P20309,1.0,[H][C@@]12C[C@@H]([C@H](OC(C)=O)[C@@]1(C)CC[C@...,MTLHNNSTTSPLFPNISSSWIHSPSDAGLPPGTVTHFGSYNVSRAA...
4,4,15145,15167,DB06770,P05177,1.0,OCC1=CC=CC=C1,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...
...,...,...,...,...,...,...,...,...
5492,5492,25690,25719,DB08411,Q6UX15,0.0,[H][C@@](C)(NC(=O)CCC[P@](O)(=O)OC1=CC=C(C=C1)...,MRPGTALQAVLLAVLLVGLRAATGRLLSASDLDLRGGQPVCRGGTQ...
5493,5493,22340,22368,DB00397,P08588,1.0,[H][C@@](C)(N)[C@]([H])(O)C1=CC=CC=C1,MGAGVLVLGASEPGNLSSAAPLPDGAATAARLLVPASPPASLLPPA...
5494,5494,322,323,DB05271,P08908,1.0,CCCN(CCC1=CC=CS1)[C@H]1CCC2=C(O)C=CC=C2C1,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...
5495,5495,24116,24145,DB05107,P01308,0.0,[H][C@@]12C[C@@H](Br)C(=O)[C@@]1(C)CC[C@@]1([H...,MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGER...


In [ ]:
train[:10000].to_csv('train_new.csv',index=False)
valid[:1400].to_csv('valid_new.csv',index=False)
test[:2700].to_csv('test_new.csv',index=False)
unlabel=pd.concat([train[10000:],valid[1400:],test[2700:]])
unlabel.reset_index(drop=True,inplace=True)
unlabel.to_csv('unlabel_new.csv',index=False)

In [ ]:
class Cus_DataLoader:
    def __init__(self, dataloader1, dataset, model, device, confidence, ratio, batch_size):
        self.dataloader1 = dataloader1

        self.label_data(dataloader1, dataset, model, device, confidence, ratio, batch_size)

        self.len_labeled = len(self.dataloader1)
        self.len_unlabeled = len(self.dataloader2)
        self.iter_unlabeled = iter(self.dataloader2)
        self.idx_labeled = 0
        self.idx_unlabeled = 0

    def label_data(self, dataloader, dataset, model, device, confidence, ratio, batch_size):
        print("Labeling unlabeled data...")

        # label with unaugmented unlabeld dataset
        data_size = len(dataset)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        model.eval()
        count = [0] * 2
        table1 = []
        table0 = []
        for idx, (d, p, d_mask, p_mask, label) in enumerate(dataloader):
          x = x.to(device)
          with torch.no_grad():
            logit = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
            m = torch.nn.Sigmoid()
            score = torch.squeeze(m(logit))

            for i, score in enumerate(score):
              if score >= 0.5:
                if score >= float(confidence):
                  table1.append([idx*batch_size + i, 1, score])
                  count[1] += 1
              else:
                if 1-score >= float(confidence):
                  table0.append([idx*batch_size + i, 0, score])
                  count[0] += 1

        print(count)
        count_dif = len(count[1])-len(count[0])

        if abs(count_dif) > 0.1*data_size:
          if count_dif>0:
            table1 =sorted(table1, key=lambda x:x[2],reverse=True)
            table1=table1[:len(count[0])]
          else:
            table0 =sorted(table0, key=lambda x:x[2],reverse=True)
            table0=table0[:len(count[1])]

        self.table = []
        for t in table1:
            for e in t:
              self.table.append(e[0:2])
        for t in table0:
            for e in t:
              self.table.append(e[0:2])

        random.seed(42)
        random.shuffle(self.table)

        training_set = BIN_Data_Encoder(self.table[0], self.table[1], dataset)
        self.dataloader2 = DataLoader(training_set, batch_size=int(batch_size*ratio), shuffle=True)

        print("Labeling completed")

    def __len__(self):
        return self.len_labeled

    def __iter__(self):
        self.idx_labeled = 0
        self.iter_labeled = iter(self.dataloader1)
        return self

    def __next__(self):
        if self.idx_labeled >= self.len_labeled:
            raise StopIteration

        d1, p1, d_mask1, p_mask1, label1 = next(self.iter_labeled)

        if self.idx_unlabeled >= self.len_unlabeled:
            self.idx_unlabeled = 0
            self.iter_unlabeled = iter(self.dataloader2)
        d2, p2, d_mask2, p_mask2, label2 = next(self.iter_unlabeled)

        d = torch.cat((d1, d2), dim=0)
        p = torch.cat((p1, p2), dim=0)
        d_mask = torch.cat((d_mask1, d_mask2), dim=0)
        p_mask = torch.cat((p_mask1, p_mask2), dim=0)
        label = torch.cat((label1, label2), dim=0)

        self.idx_labeled += 1
        self.idx_unlabeled += 1

        return d, p, d_mask, p_mask, label

#2.datasetting2

In [ ]:
df1=pd.read_csv('bindingDB/train.csv')
df2=pd.read_csv('bindingDB/val.csv')
df3=pd.read_csv('bindingDB/test.csv')

In [ ]:
df1

,Unnamed: 0,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,0,Cc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,[0. 0. 0. ... 0. 0. 0.],[5. 2.692 3.846 ... 0. 0. 0. ]
1,1,COc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,[0. 0. 0. ... 0. 0. 0.],[5. 2.692 3.846 ... 0. 0. 0. ]
2,2,NS(=O)(=O)c1ccc(s1)S(=O)(=O)NCc1cccs1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,[0. 0. 0. ... 0. 0. 0.],[5. 2.692 3.846 ... 0. 0. 0. ]
3,3,NS(=O)(=O)c1cc2C(O)CN(Cc3cccs3)S(=O)(=O)c2s1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,[0. 0. 0. ... 0. 0. 0.],[5. 2.692 3.846 ... 0. 0. 0. ]
4,4,COc1ccc(cc1)N1CC(O)c2cc(sc2S1(=O)=O)S(N)(=O)=O,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,[0. 0. 0. ... 0. 0. 0.],[5. 2.692 3.846 ... 0. 0. 0. ]
...,...,...,...,...,...,...
12663,12663,Cn1c(Nc2ccc(cc2)C(F)(F)F)nc2cc(Oc3ccnc(c3)-c3n...,MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...,0,[0. 0. 0. ... 0. 0. 0.],[5.664 5.752 5.752 ... 0. 0. 0. ]
12664,12664,COCCOc1cc2ncnc(Nc3cccc(c3)C#C)c2cc1OCCOC,MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQ...,0,[0. 0. 0. ... 0. 0. 0.],[6.107 4.58 3.308 ... 0. 0. 0. ]
12665,12665,CC(C)N1C(=O)CC(C)(C)c2cc(ccc12)C#Cc1ccc(cc1)C(...,MASNSSSCPTPGGGHLNGYPVPPYAFFFPPMLGGLSPPGALTTLQH...,0,[0. 1. 0. ... 0. 0. 0.],[4.545 5.195 3.247 ... 0. 0. 0. ]
12666,12666,C[C@@H](O)COc1cn2ncnc(Oc3ccc4[nH]c(C)cc4c3F)c2c1C,MALRRLGAALLLLPLLAAVEETLMDSTTATAELGWMVHPPSGWEEV...,0,[0. 1. 0. ... 0. 0. 1.],[6.919 5.782 4.645 ... 0. 0. 1. ]


In [ ]:
smiles=list(set(np.concatenate((df1['SMILES'].values,df2['SMILES'].values,df3['SMILES'].values))))

In [ ]:
sequence=list(set(np.concatenate((df1['Target Sequence'].values,df2['Target Sequence'].values,df3['Target Sequence'].values))))

In [ ]:
len(smiles)

7165

In [ ]:
len(sequence)

1254

In [ ]:
all=[]
for i in smiles:
  for j in sequence:
    all.append([i,j])

In [ ]:
len(all)

8984910

In [ ]:
random.shuffle(all)

In [ ]:
new_df=pd.DataFrame(all,columns=['SMILES','Target Sequence'])

In [ ]:
new_df['Label']=0

In [ ]:
new_df.to_csv('unlabel_new.csv',index=False)

In [ ]:
unlabel=new_df[:2000000]

In [ ]:
unlabel.to_csv('unlabel_new.csv',index=False) #unlabel

In [ ]:
type(df2['SMILES'].values)

numpy.ndarray

In [ ]:
def pre_label(model,dataset,batch_size,confidence):
  model.cuda()
  model.eval()
  count = [0] * 2
  table1 = []
  table0 = []
  dataset_encoded = BIN_Data_Encoder(dataset.index.values, dataset.Label.values, dataset)
  dataloader = DataLoader(dataset_encoded, batch_size=batch_size, shuffle=False, drop_last=True)

  with torch.no_grad():
    for idx, (d, p, d_mask, p_mask, label) in tqdm(enumerate(dataloader),desc="inference"):
      logit = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

      m=torch.nn.Sigmoid()
      score=torch.squeeze(m(logit))
      score=score.detach().cpu().numpy()
      for i, score in enumerate(score):
        if score >= 0.5:
          if score >= float(confidence):
            table1.append([idx*batch_size + i, 1, score])
            count[1] += 1
        else:
          if 1-score >= float(confidence):
            table0.append([idx*batch_size + i, 0, score])
            count[0] += 1

  table1 =sorted(table1, key=lambda x:x[2],reverse=True)
  table0 =sorted(table0, key=lambda x:x[2],reverse=False)

  if count[1]>=1000000:
    table1=table1[:1000000]
  if count[0]>=1000000:
    table0=table0[:1000000]

  diff=count[0]-count[1]
  if diff>=0:
    table0=table0[:count[1]]
  else:
    table1=table1[:count[0]]

  print(str(count[0])+' / '+str(count[1]))
  print(str(len(table0))+' / '+str(len(table1)))
  table = []
  for t in table1:
    table.append(t[0:2])
  for t in table0:
    table.append(t[0:2])

  random.seed(42)
  random.shuffle(table)

  model.cpu()
  del model
  torch.cuda.empty_cache()
  return table

In [ ]:
model=torch.load('model_max.pt')
model.batch_size=64

<ipython-input-13-4a5df7fcdf5c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load('model_max.pt')


In [ ]:
new_df

,SMILES,Target Sequence,Label
0,CC(C)C[C@@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)[C@@H...,MSDGLDNEEKPPAPPLRMNSNNRDSSALNHSSKPLPMAPEEKNKKA...,0
1,OC[C@H]1O[C@H](OC[C@@H](O)[C@@H](O)[C@H](O)Cn2...,MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTV...,0
2,CC1CCC(C[C@H](CC(=O)N2CCOCC2)C(=O)N[C@@]2(CCN(...,MGQELCAKTVQPGCSCYHCSEGGEAHSCRRSQPETTEAAFKLTDLK...,0
3,COC(=O)c1ccccc1Oc1nc(NC(C)C)nc(NC(C)C)n1,MSSASGLRRGHPAGGEENMTETDAFYKREMFDPAEKYKMDHRRRGI...,0
4,CSC1=NC(C(C(=O)OC(C)C)=C(C)N1)c1cccc(c1)[N+]([...,MQWAVGRRWLWVALFLAAVAVLTQIVWLWLGTQNFVFQREEIAQLA...,0
...,...,...,...
8984905,CC[C@H](C)[C@@H]1N(C)C(=O)CN(C)C(=O)[C@H](CC)N...,MPPYISAFQAAYIGIEVLIALVSVPGNVLVIWAVKVNQALRDATFC...,0
8984906,COc1cc2ncnc(Sc3cccc(NC(=O)Nc4cc(nn4-c4ccccc4)C...,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...,0
8984907,COc1ccccc1N1CCN(CC2CC(=O)c3ccccc23)CC1,MDKLDANVSSKEGFGSVEKVVLLTFLSAVILMAILGNLLVMVAVCR...,0
8984908,Cc1ccc(SCC(=O)N2CCOCC2)cc1,MAAAAAAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNLNK...,0


In [ ]:
len(new_df['SMILES'].values)

8984910

In [ ]:
new_df.loc[10,'SMILES']

'COc1cc2ncnc(Oc3cccc(NC(=O)Nc4cc(on4)C(C)(C)C(F)(F)F)c3)c2cc1O'

In [ ]:
result=pre_label(model,new_df,64,0.52)
df_result=pd.DataFrame(result,columns=['index','Label'])
for i in range(len(df_result)):
  df_result.loc[i,'SMILES']=new_df['SMILES'].values[df_result['index'].values[i]]
  df_result.loc[i,'Target Sequence']=new_df['Target Sequence'].values[df_result['index'].values[i]]
df_result.to_csv('result.csv',index=False)

from google.colab import runtime
runtime.unassign()

inference: 0it [00:00, ?it/s]

3186349 / 5306091
1000000 / 1000000


#3.datasetting3

In [ ]:
import os
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/mini/unlabel_new.csv')

In [ ]:
df0=df[df['Label']==0].copy()

In [ ]:
df1=df[df['Label']==1].copy()

In [ ]:
ratio1=0.85
ratio2=0.90
ratio3=0.95
ratio4=1

In [ ]:
df0_shuffle = df.sample(frac=1).reset_index(drop=True)
df0_shuffle1 = df0_shuffle.copy()
df0_shuffle2 = df0_shuffle.copy()
df0_shuffle3 = df0_shuffle.copy()

In [ ]:
df1_shuffle = df1.sample(frac=1).reset_index(drop=True)
df1_shuffle1 = df1_shuffle.copy()
df1_shuffle2 = df1_shuffle.copy()
df1_shuffle3 = df1_shuffle.copy()

In [ ]:
df0_shuffle1.iloc[:int(len(df0_shuffle1)*(1-ratio1))]=0
df0_shuffle2.iloc[:int(len(df0_shuffle2)*(1-ratio2))]=0
df0_shuffle3.iloc[:int(len(df0_shuffle3)*(1-ratio3))]=0

In [ ]:
df1_shuffle1.iloc[:int(len(df1_shuffle1)*(1-ratio1))]=0
df1_shuffle2.iloc[:int(len(df1_shuffle2)*(1-ratio2))]=0
df1_shuffle3.iloc[:int(len(df1_shuffle3)*(1-ratio3))]=0

In [ ]:
unlabel_acc_1=pd.concat([df0_shuffle1,df1_shuffle1]).sample(frac=1).reset_index(drop=True)
unlabel_acc_2=pd.concat([df0_shuffle2,df1_shuffle2]).sample(frac=1).reset_index(drop=True)
unlabel_acc_3=pd.concat([df0_shuffle3,df1_shuffle3]).sample(frac=1).reset_index(drop=True)

In [ ]:
unlabel_acc_1.to_csv('/content/drive/MyDrive/mini/unlabel_acc_1.csv',index=False)
unlabel_acc_2.to_csv('/content/drive/MyDrive/mini/unlabel_acc_2.csv',index=False)
unlabel_acc_3.to_csv('/content/drive/MyDrive/mini/unlabel_acc_3.csv',index=False)

#4.testing

In [ ]:
model.cuda()
model.eval()
count = [0] * 2
table1 = []
table0 = []
dataset=new_df
batch_size=64
dataset_encoded = BIN_Data_Encoder(dataset.index.values, dataset.Label.values, dataset)
dataloader = DataLoader(dataset_encoded, batch_size=batch_size, shuffle=False, drop_last=True)

d, p, d_mask, p_mask, label = next(iter(dataloader))
logit = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

In [ ]:
m=torch.nn.Sigmoid()
score=torch.squeeze(m(logit))
score=score.detach().cpu().numpy()
confidence=0.52
for i, score in enumerate(score):
  if score >= 0.5:
    if score >= float(confidence):
      table1.append([i, 1, score])
      count[1] += 1
  else:
    if 1-score >= float(confidence):
      table0.append([i, 0, score])
      count[0] += 1